In [181]:
import numpy as np
import pandas as pd

In [182]:
mail_data = pd.read_csv('/content/mail_data.csv')

In [183]:
mail_data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [184]:
mail_data.shape

(5572, 2)

In [185]:
#knowing is there any null values..
mail_data.isnull().sum()

,0
Category,0
Message,0


In [186]:
#replace the null values with a null string..
mail_data = mail_data.where((pd.notnull(mail_data)),'')

In [187]:
#Making the category "Spam" = 0 and "Ham"=1
mail_data.loc[mail_data['Category']=='spam','Category'] = 0
mail_data.loc[mail_data['Category']=='ham','Category'] = 1

In [188]:
mail_data.head()

,Category,Message
0,1,"Go until jurong point, crazy.. Available only ..."
1,1,Ok lar... Joking wif u oni...
2,0,Free entry in 2 a wkly comp to win FA Cup fina...
3,1,U dun say so early hor... U c already then say...
4,1,"Nah I don't think he goes to usf, he lives aro..."


In [189]:
#X = mail_data.drop(columns='Category',axis=1)
#OR
X = mail_data['Message']   ## because there is only 2 columns..

In [190]:
print(X)

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: Message, Length: 5572, dtype: object


In [191]:
Y = mail_data['Category']

**Splitting the Data into Training Data and Testing Data**

In [192]:
from sklearn.model_selection import train_test_split

In [193]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=3)

In [194]:
print(X.shape,X_train.shape,X_test.shape)

(5572,) (4457,) (1115,)


**Feature Extraction**

In [195]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [196]:
# transform the text data to feature vectors that can be used as input to the Logistic regression

feature_extraction = TfidfVectorizer(min_df = 1, stop_words='english', lowercase=True)

X_train_features = feature_extraction.fit_transform(X_train)
X_test_features = feature_extraction.transform(X_test)

# convert Y_train and Y_test values as integers

Y_train = Y_train.astype('int')
Y_test = Y_test.astype('int')

In [197]:
print(X_train_features)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 34775 stored elements and shape (4457, 7431)>
  Coords	Values
  (0, 2329)	0.38783870336935383
  (0, 3811)	0.34780165336891333
  (0, 2224)	0.413103377943378
  (0, 4456)	0.4168658090846482
  (0, 5413)	0.6198254967574347
  (1, 3811)	0.17419952275504033
  (1, 3046)	0.2503712792613518
  (1, 1991)	0.33036995955537024
  (1, 2956)	0.33036995955537024
  (1, 2758)	0.3226407885943799
  (1, 1839)	0.2784903590561455
  (1, 918)	0.22871581159877646
  (1, 2746)	0.3398297002864083
  (1, 2957)	0.3398297002864083
  (1, 3325)	0.31610586766078863
  (1, 3185)	0.29694482957694585
  (1, 4080)	0.18880584110891163
  (2, 6601)	0.6056811524587518
  (2, 2404)	0.45287711070606745
  (2, 3156)	0.4107239318312698
  (2, 407)	0.509272536051008
  (3, 7414)	0.8100020912469564
  (3, 2870)	0.5864269879324768
  (4, 2870)	0.41872147309323743
  (4, 487)	0.2899118421746198
  :	:
  (4454, 2855)	0.47210665083641806
  (4454, 2246)	0.47210665083641806
  (4455, 4456)	0.24

**Model Evaluation**

In [198]:
from sklearn.metrics import precision_score,f1_score,accuracy_score,recall_score

In [199]:
def model_evaluation(true,predicted):
  acc = accuracy_score(true,predicted)
  ps = precision_score(true,predicted)
  f1 = f1_score(true,predicted)
  rs = recall_score(true,predicted)
  return acc,ps,f1,rs

**Model Selection**

In [200]:
from sklearn.model_selection import cross_val_score

In [201]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [202]:
models = [LogisticRegression(max_iter=1000),SVC(),KNeighborsClassifier(),DecisionTreeClassifier(),RandomForestClassifier()]

In [203]:
for model in models:

  #training the data..
  model.fit(X_train_features,Y_train)

  #Making Predictions
  Y_train_pred = model.predict(X_train_features)
  Y_test_pred = model.predict(X_test_features)

  #Evaluate Train Data
  model_train_acc,model_train_ps,model_train_f1,model_train_rs = model_evaluation(Y_train_pred,Y_train)

  #Evaluate Test Data
  model_test_acc,model_test_ps,model_test_f1,model_test_rs = model_evaluation(Y_test_pred,Y_test)

  print(f"Model Name : {model}")
  print()

  print(f"Train Data Accuracy : {model_train_acc}")
  print(f"Train Data Precision : {model_train_ps}")
  print(f"Train Data Recall : {model_train_rs}")
  print(f"Train Data F1 Score : {model_train_f1}")

  print("------------------------------------------------------------------")

  print(f"Test Data Accuracy : {model_test_acc}")
  print(f"Test Data Precision : {model_test_ps}")
  print(f"Test Data Recall : {model_test_rs}")
  print(f"Test Data F1 Score :{model_test_f1}")

  print("------------------------------------------------------------------")

  cv_score = cross_val_score(model,X_train_features,Y_train,cv=5)
  print("Cross Validation=",cv_score)
  print(f"Cross Mean Validation Score : {np.mean(cv_score)}")
  print()

  print("------------------------------------------------------------------")
  print("------------------------------------------------------------------")
  print()


Model Name : LogisticRegression(max_iter=1000)

Train Data Accuracy : 0.9676912721561588
Train Data Precision : 0.9989650711513584
Train Data Recall : 0.9650087478130467
Train Data F1 Score : 0.9816933638443935
------------------------------------------------------------------
Test Data Accuracy : 0.9668161434977578
Test Data Precision : 1.0
Test Data Recall : 0.9628886659979939
Test Data F1 Score :0.9810935104752172
------------------------------------------------------------------
Cross Validation= [0.93609865 0.93834081 0.94163861 0.94276094 0.94276094]
Cross Mean Validation Score : 0.9403199911421138

------------------------------------------------------------------
------------------------------------------------------------------

Model Name : SVC()

Train Data Accuracy : 0.99798070450976
Train Data Precision : 1.0
Train Data Recall : 0.9976768198244709
Train Data F1 Score : 0.9988370590515571
------------------------------------------------------------------
Test Data Accuracy 

**HyperParameter Tuning**

In [204]:
hyperparameters = {
    'svc': {
        'kernel':['linear','poly','rbf','sigmoid'],
        'C':[1,5,10,20]
    },
    'decision_tree': {
        'criterion':['gini','entropy'],
        'max_depth':[3,6,9,11]
    },
    'random_forest': {
        'n_estimators':[10,20,30,40,50],
        'criterion':['gini','entropy'],
        'max_depth':[3,6,9,11]
    }
}


In [205]:
model_keys = list(hyperparameters.keys())

In [206]:
models = [SVC(),DecisionTreeClassifier(),RandomForestClassifier()]

**Applying RandomizedSearchCV**

In [207]:
from sklearn.model_selection import RandomizedSearchCV

In [208]:
def ModelSelection(models_list, hyperparameters, model_keys, X_train_features, Y_train):
    result = []
    i = 0

    for model in models_list:
        key = model_keys[i]
        params = hyperparameters[key]
        i += 1

        print(model)
        print(params)
        print("----------------------------------")

        classifier = RandomizedSearchCV(model, params, cv=5)
        classifier.fit(X_train_features, Y_train)

        result.append({
            'Model Used:': model,
            'Best Parameters': classifier.best_params_,
            'Best Score': classifier.best_score_
        })

    result_df = pd.DataFrame(result, columns=['Model Used:', "Best Parameters", "Best Score"])
    return result_df


In [209]:
ModelSelection(models,hyperparameters,model_keys,X_train_features,Y_train)

SVC()
{'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [1, 5, 10, 20]}
----------------------------------
DecisionTreeClassifier()
{'criterion': ['gini', 'entropy'], 'max_depth': [3, 6, 9, 11]}
----------------------------------


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomForestClassifier()
{'n_estimators': [10, 20, 30, 40, 50], 'criterion': ['gini', 'entropy'], 'max_depth': [3, 6, 9, 11]}
----------------------------------


,Model Used:,Best Parameters,Best Score
0,SVC(),"{'kernel': 'linear', 'C': 5}",0.978011
1,DecisionTreeClassifier(),"{'max_depth': 11, 'criterion': 'gini'}",0.952884
2,RandomForestClassifier(),"{'n_estimators': 10, 'max_depth': 11, 'criteri...",0.906215


**Selected Model : SVC() **

In [210]:
model = SVC(C=5,kernel='linear')

In [211]:
model.fit(X_train_features,Y_train)

SVC(C=5, kernel='linear')

In [212]:
# Predict a new email message using the trained model and vectorizer

input_data = ["I've been searching for the right words to thank you for this breather. I promise I won't take your help for granted and will fulfil my promise. You have been wonderful and a blessing at all times"]

# DO NOT use fit_transform here, only transform using trained vectorizer
input_data_features = feature_extraction.transform(input_data)

# Predict using the already trained model
prediction = model.predict(input_data_features)
print("Prediction:", prediction)

if prediction[0] == 1:
    print("Ham mail")
else:
    print("Spam mail")


Prediction: [1]
Ham mail


In [213]:
import pickle

In [215]:
filename = 'spam_mail_model.sav'
pickle.dump(model,open(filename,'wb'))

In [218]:
loading_model = pickle.load(open('/content/spam_mail_model.sav','rb'))

In [219]:
# Predict a new email message using the trained model and vectorizer

input_data = ["I've been searching for the right words to thank you for this breather. I promise I won't take your help for granted and will fulfil my promise. You have been wonderful and a blessing at all times"]

# DO NOT use fit_transform here, only transform using trained vectorizer
input_data_features = feature_extraction.transform(input_data)

# Predict using the already trained model
prediction = loading_model.predict(input_data_features)
print("Prediction:", prediction)

if prediction[0] == 1:
    print("Ham mail")
else:
    print("Spam mail")

Prediction: [1]
Ham mail
